In [ ]:
import argparse
import os
import numpy as np
from tqdm import tqdm

from mypath import Path
from dataloaders import make_data_loader
from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *
from utils.loss import SegmentationLosses
from utils.calculate_weights import calculate_weigths_labels
from utils.lr_scheduler import LR_Scheduler
from utils.saver import Saver
from utils.summaries import TensorboardSummary
from utils.metrics import Evaluator

In [ ]:
model = DeepLab(num_classes=2, backbone='xception', output_stride=16, sync_bn=False, freeze_bn=False)

In [ ]:
import torch
# run/lits_liver/deeplab-xception-liver-bce/model_best.pth.tar

ckpt = torch.load('run/lits_liver/deeplab-xception-liver-bce/model_best.pth.tar')
state_dict = ckpt['state_dict']
model.load_state_dict(state_dict)

In [ ]:
from dataloaders.datasets.lits import LiverSegmentation
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import argparse

# Define Dataloader
parser = argparse.ArgumentParser()
parser.add_argument('-f')
args = parser.parse_args()
args.base_size = 513
args.crop_size = 513

liver_test = LiverSegmentation(args, split='val')
print("num test img: ", len(liver_test))
dataloader = DataLoader(liver_test, batch_size=5, shuffle=True, num_workers=0)

device = 'cuda'
model.to(device)
model.eval()

for i, sample in enumerate(dataloader):
    image, target = sample['image'], sample['label']
    image = image.to(device)
    with torch.no_grad():
        output = model(image)
    
    pred = output.data.cpu().numpy()
    target = target.cpu().numpy()
    pred = np.argmax(pred, axis=1)
    # print(np.unique(pred))
    # print(np.unique(target))
    
    image = image.cpu().numpy()
    for idx in range(5):
        if len(np.unique(pred[idx])) > 1 or len(np.unique(target[idx])) > 1:
            print(i, idx)
            pred_ = pred[idx].astype(np.uint8)
            pred_[pred_ != 0] = 255

            target_ = target[idx].astype(np.uint8)
            target_[target_ != 0] = 255

            img_tmp = np.transpose(image[idx], axes=[1, 2, 0])
            img_tmp *= (0.229, 0.224, 0.225)
            img_tmp += (0.485, 0.456, 0.406)
            img_tmp *= 255.0
            img_tmp = img_tmp.astype(np.uint8)

            display(Image.fromarray(pred_))
            display(Image.fromarray(target_))
            display(Image.fromarray(img_tmp))
    

In [ ]:
np.unique(target[1])

In [ ]:
from PIL import Image
idx = 3

pred_ = pred[idx].astype(np.uint8)
pred_[pred_ != 0] = 255

target_ = target[idx].astype(np.uint8)
target_[target_ != 0] = 255

# image = image.cpu().numpy()
img_tmp = np.transpose(image[idx], axes=[1, 2, 0])
img_tmp *= (0.229, 0.224, 0.225)
img_tmp += (0.485, 0.456, 0.406)
img_tmp *= 255.0
img_tmp = img_tmp.astype(np.uint8)

display(Image.fromarray(pred_))
display(Image.fromarray(target_))
Image.fromarray(img_tmp)

In [ ]:
# image = image.cpu().numpy()
img_tmp = np.transpose(image[3], axes=[1, 2, 0])
img_tmp *= (0.229, 0.224, 0.225)
img_tmp += (0.485, 0.456, 0.406)
img_tmp *= 255.0
img_tmp = img_tmp.astype(np.uint8)

In [ ]:
Image.fromarray(img_tmp)